In [8]:
import os
import pandas as pd
from ebmdatalab import bq

In [ ]:
sql = f"""
SELECT 
i.month, -- Converts TIMESTAMP to DATE (as it's always just 1st of month)
regional_team,
bnf_code,
bnf_name,
CASE -- This uses the BNF hierarchy to classify by drug
  WHEN bnf_code LIKE '0601023AZ%' THEN 'Tirzepatide'
  -- Add others here as needed
END AS drug,
SUM(items) AS items,
SUM(quantity) AS quantity,

FROM 
  ebmdatalab.hscic.normalised_prescribing_standard AS i
INNER JOIN
  ebmdatalab.hscic.practices prac
ON
  practice = prac.code
  AND setting = 4

LEFT JOIN (
  SELECT
    month,
    pct_id,
    SUM(total_list_size) AS denominator
  FROM
    hscic.practice_statistics_all_years
  GROUP BY
    month,
    pct_id) AS s
ON
  i.month = s.month 
  AND
  i.practice = s.pct_id
WHERE
i.month >= '2020-01-01'
AND
bnf_code LIKE '0601023AZ%' -- Tirzepatide
GROUP BY -- Needed as we are selecting SUM of items and quantity
i.month,
regional_team,
bnf_code,
bnf_name  

"""

# Define the CSV path for caching the results.
csv_path = os.path.join('..', 'data', 'tirzepatide_lists.csv')

# Use the cached_read function from the bq library to run the query.
vtm_matched = bq.cached_read(sql, csv_path=csv_path)